In [1]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created (in the wow folder):
- **wow.log.Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **wow.log.Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **wow.query.Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **wow.query.Predicate**
  - *A Collection of functions that will be applied to those iterators*

There are also another 2 modules in addition to both modules containing classes above:
- **ui**
  - contain ui elements in this notebook

- **fights**
  - contains scripts to analyze each fight (only nerzhul for now)


In [2]:
# Initialize
import wow.ui as ui
import wow.helper as help

from wow.query import Query, Predicate
from wow.log import Record, Encounter

# (z, f) = ui.import_file()


# Create Data Structures

In [3]:
# help.unzip('WoWCombatLog-012722_214646.zip', 'wow')

In [4]:
file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\RaiderIOLogsArchive\WoWCombatLog-012722_214646.txt'
# file = r'C:\Program Files (x86)\World of Warcraft\_retail_\Logs\WoWCombatLog-013122_220632.txt'

(log, encounters) = Encounter.parse(file)


Encounters:  20


# Detailed Analysis

In [5]:
# Define Encounter being Analysed
encSelect = ui.pick_encounter(encounters)

In [29]:
e = encounters[encSelect.value]
# Query(e.players).groupby(
#   lambda x: x[1],
#   lambda x: x[0],
# ).list()


In [30]:
e.actions('SPELL_AURA_APPLIED', '"Sorrowful Procession"').groupby(
    lambda x: x[1],
    lambda x: x[2],
).dict()


{'Creature-0-4216-2450-10260-177117-0000735A55': ['"Spartanovix-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735A56': ['"Djøw-Azralon"',
  '"Thilokz-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735A7D': ['"Djøw-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735A7E': ['"Thilokz-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AAB': ['"Djøw-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AAD': ['"Thilokz-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AD4': ['"Spartanovix-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AD6': ['"Thilokz-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AD7': ['"Djøw-Azralon"'],
 'Creature-0-4216-2450-10260-177117-0000735AF9': ['"Jathäl-Azralon"']}

In [31]:
e.hostile_action.groupby(
    lambda x: x[0],
    lambda x: x[1],
    set
).dict()


{'"Helm of Suffering"': {'SPELL_ABSORBED',
  'SPELL_CAST_START',
  'SPELL_CAST_SUCCESS',
  'SPELL_DAMAGE',
  'SPELL_MISSED'},
 '"Malicious Gauntlet"': {'SPELL_CAST_SUCCESS'},
 '"Orb of Torment"': {'SPELL_ABSORBED',
  'SPELL_AURA_APPLIED',
  'SPELL_AURA_REFRESH',
  'SPELL_AURA_REMOVED',
  'SPELL_CAST_START',
  'SPELL_CAST_SUCCESS',
  'SPELL_PERIODIC_DAMAGE',
  'SPELL_PERIODIC_MISSED'},
 '"Soulrender Dormazain"': {'SPELL_AURA_APPLIED'}}

In [9]:
# import humanize
# from pympler import asizeof
# humanize.naturalsize(
# )
# asizeof.asizeof(log, limit=6, stats=3, cutoff=0)

In [32]:
e.end

1/27 23:55:33.523: ENCOUNTER_END entry: 1050994
ENCOUNTER_END,2432,"Remnant of Ner'zhul",16,20,0,219706

In [36]:
e.q.filter(
    Predicate.isCreatureAction()
).filter(
    Predicate.isActorHostile()
).map(
    ( Predicate.getActionId(), Predicate.getAction(), Predicate.getActor(), Predicate.getEvent())
).qset(
).sort(
    lambda x: x[1]
).list()


[('355790', '"Eternal Torment"', '"Orb of Torment"', 'SPELL_CAST_SUCCESS'),
 ('355790', '"Eternal Torment"', '"Orb of Torment"', 'SPELL_AURA_REMOVED'),
 ('355790', '"Eternal Torment"', '"Orb of Torment"', 'SPELL_AURA_APPLIED'),
 ('350694', '"Grasp of Malice"', '"Malicious Gauntlet"', 'SPELL_CAST_SUCCESS'),
 ('108446', '"Soul Link"', '"Orb of Torment"', 'SPELL_ABSORBED'),
 ('357812', '"Suffering"', '"Helm of Suffering"', 'SPELL_MISSED'),
 ('349891', '"Suffering"', '"Helm of Suffering"', 'SPELL_CAST_SUCCESS'),
 ('349891', '"Suffering"', '"Helm of Suffering"', 'SPELL_CAST_START'),
 ('357812', '"Suffering"', '"Helm of Suffering"', 'SPELL_ABSORBED'),
 ('357812', '"Suffering"', '"Helm of Suffering"', 'SPELL_DAMAGE'),
 ('350073', '"Torment"', '"Orb of Torment"', 'SPELL_PERIODIC_MISSED'),
 ('350071', '"Torment"', '"Orb of Torment"', 'SPELL_CAST_START'),
 ('350071', '"Torment"', '"Orb of Torment"', 'SPELL_CAST_SUCCESS'),
 ('350073', '"Torment"', '"Orb of Torment"', 'SPELL_AURA_REFRESH'),
 ('352